In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import wget
import os
from datetime import datetime
import requests

In [6]:
# baixa preços cripto
def crip_quotation(symbol, ini_date, comparison_symbol='BRL', all_data=True, limit=1, aggregate=1, exchange=''):
    from datetime import datetime
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.fromtimestamp(d) for d in df.time]
    
    df['timestamp'] = df['timestamp'].dt.normalize()
    df = df.set_index('timestamp').drop(['time','high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    df = df[ini_date:]
    
    return df

In [9]:
# recebe caminho de um arquivo como parâmetro, devolve dataframe com papeis e cotacoes
def baixa_cotacoes(arquivo,cripto=False):
    df = pd.read_csv(arquivo, delimiter=";")
    df["ini"] = pd.to_datetime(df["ini"], format="%d/%m/%Y")
    df.set_index("acao", inplace=True)
    cotacoes = pd.DataFrame()
    for acao in df.index:
        ini = df.loc[acao,"ini"]
        print(acao, ini)
        if cripto == True:
            cotacoes[acao] = crip_quotation(acao, ini_date=ini)["close"]
        elif cripto == False:
            cotacoes[acao] = yf.download(acao, start=ini)["Adj Close"] # +".SA"
    return(cotacoes)

In [13]:
# baixa renda fixa
def precos_rf():
    df = pd.DataFrame()
    rf = pd.read_excel('lista rf.xlsx')
    for i in range(0,len(rf)):
        link = rf['download'][i] # link para download do arquivo de rf
        arquivo = rf['arquivo'][i]
        aba = rf['ativo'][i] # o nome da aba é o nome do ativo
        ini_date = rf['data ini'][i]
        # baixa dados
        wget.download(link)
        td = pd.read_excel(arquivo, aba,skiprows=[0]).set_index('Dia') # skiprows é uma lista de linhas para pular. No caso pula a linha 0
        os.remove(arquivo) # deleta depois de baixar
        td = td[ini_date:] # filtra só as linhas a partir da data desejada
        # transforma pro formato datetime
        td = td.set_index(pd.DataFrame([datetime.strptime(date, '%d/%m/%Y').date() for date in td.index[:]])[0]) 
        td = td['PU Compra Manhã']  # filtra a coluna de preço correta
        df[aba] = td # cria uma coluna com o nome do ativo e o preço histórico
    return(df)

In [10]:
df = baixa_cotacoes("lista acoes.csv")
df.to_excel("./resultados/cotacoes acao.xlsx")
df

VALE3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
KLBN11.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
MOVI3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
PNVL3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
RRRP3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
TTEN3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
LAVV3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
ALSO3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
YDUQ3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
TUPY3.SA 2022-05-30 00:00:00
[*********************100%***********************]  1 of 1 completed
MGLU3.SA 2022-05-30

,VALE3.SA,KLBN11.SA,MOVI3.SA,PNVL3.SA,RRRP3.SA,TTEN3.SA,LAVV3.SA,ALSO3.SA,YDUQ3.SA,TUPY3.SA,MGLU3.SA,BBAS3.SA
Date,,,,,,,,,,,,
2022-05-30,86.650002,21.724131,17.461679,11.64,49.310001,9.41,5.28,19.219999,16.540001,22.420000,3.84,35.802589
2022-05-31,86.209999,21.861563,17.669323,11.53,48.750000,9.60,5.14,19.309999,16.180000,23.100000,3.72,36.348511
2022-06-01,88.239998,22.273861,17.857187,11.36,48.770000,9.75,5.13,18.660000,16.450001,23.469999,3.71,36.417992
2022-06-02,89.900002,22.332760,17.698984,11.48,48.560001,9.40,5.18,19.110001,16.650000,24.360001,3.80,36.447773
2022-06-03,88.459999,21.792849,16.680553,11.33,48.599998,9.36,5.18,18.920000,15.720000,24.379999,3.59,35.941551
2022-06-06,88.550003,21.832113,16.324594,10.99,46.860001,8.96,5.09,18.600000,15.170000,23.830000,3.40,35.494888
2022-06-07,90.620003,21.822296,16.364145,10.76,46.860001,8.92,5.00,18.459999,15.180000,23.219999,3.29,35.246742
2022-06-08,87.500000,21.537617,15.503916,10.67,46.790001,8.81,4.90,18.670000,14.920000,23.430000,3.22,34.929111
2022-06-09,84.540001,21.390368,15.760996,10.55,46.020000,8.83,4.98,18.290001,14.990000,23.590000,3.01,34.879486


In [11]:
df = baixa_cotacoes("lista acoes US.csv")
df.to_excel("./resultados/cotacoes acao US.xlsx")
df

SPY 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
AAXJ 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
XLU 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed


,SPY,AAXJ,XLU
Date,,,
2022-05-02,412.697266,70.681290,69.970062
2022-05-03,414.589111,71.258598,70.237923
2022-05-04,427.214569,71.945404,71.805374
2022-05-05,412.030151,69.247948,71.071251
2022-05-06,409.570770,68.292397,71.636726
...,...,...,...
2022-07-27,401.040009,68.519997,70.779999
2022-07-28,406.070007,68.550003,73.320000
2022-07-29,411.989990,68.000000,73.949997


In [14]:
df = precos_rf()
df.to_excel("./resultados/cotacoes rf.xlsx")
df

100% [..............................................................................] 94208 / 94208

,LTN 010125,LFT 010327,NTN-B Princ 150535
0,,,
2022-05-02,735.84,11503.77,1930.89
2022-05-03,734.44,11509.38,1924.80
2022-05-04,733.90,11515.28,1906.99
2022-05-05,735.98,11521.80,1933.96
2022-05-06,731.99,11528.71,1914.91
...,...,...,...
2022-07-27,739.41,11865.10,1831.88
2022-07-28,742.31,11871.00,1842.95
2022-07-29,745.85,11877.00,1846.64


In [15]:
df = baixa_cotacoes("lista commodities.csv")
df.to_excel("./resultados/cotacoes commodities.xlsx")
df

SLV 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
GSG 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
COPX 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed
LIT 2022-05-01 00:00:00
[*********************100%***********************]  1 of 1 completed


,SLV,GSG,COPX,LIT
Date,,,,
2022-05-02,20.870001,24.059999,38.640076,65.605957
2022-05-03,20.830000,23.559999,39.060608,66.625114
2022-05-04,21.280001,24.430000,39.901672,68.843285
2022-05-05,20.780001,24.330000,37.838131,66.984818
2022-05-06,20.660000,24.370001,37.143764,66.884903
...,...,...,...,...
2022-07-27,17.600000,22.860001,28.900000,74.089996
2022-07-28,18.430000,22.790001,29.209999,74.330002
2022-07-29,18.700001,22.930000,30.490000,74.389999


In [16]:
df = baixa_cotacoes("lista hedge.csv")
df.to_excel("./resultados/cotacoes hedge.xlsx")
df

GLD 2022-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed


,GLD
Date,
2022-05-02,173.630005
2022-05-03,174.089996
2022-05-04,175.800003
2022-05-05,175.130005
2022-05-06,175.419998
...,...
2022-07-27,161.669998
2022-07-28,163.639999
2022-07-29,164.100006


In [17]:
cotacoes = baixa_cotacoes("lista cripto.csv", cripto=True)
cotacoes.to_excel("./resultados/cotacoes cripto.xlsx")
cotacoes

BTC 2022-05-02 00:00:00
ETH 2022-05-02 00:00:00
DOT 2022-05-02 00:00:00
ALGO 2022-05-02 00:00:00
MATIC 2022-05-02 00:00:00


,BTC,ETH,DOT,ALGO,MATIC
timestamp,,,,,
2022-05-02,188398.91,13890.65,73.45,3.344,5.367
2022-05-03,195713.13,14498.43,80.54,3.526,5.778
2022-05-04,185037.49,13909.27,73.77,3.214,5.383
2022-05-05,184045.43,13761.08,73.20,3.554,5.327
2022-05-06,181693.05,13497.98,70.43,3.837,5.221
...,...,...,...,...,...
2022-07-28,123822.14,8968.44,42.66,1.746,4.882
2022-07-29,123244.64,8844.04,42.77,1.758,4.846
2022-07-30,121882.78,8782.87,45.05,1.755,4.844
